<h1><center> Python project proposal </center></h1>
<h2><center>Current car prices from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters. In the end, we will perform some visualizations of distribution of price and selected parameters and try to select some underprices offers.

This task might be better to perform continuouslly, for example each hour, to not overload the website. This approach might also allow for longer time period to analyze the data. But since this project should be designed as one-time run, we decided to limit the data to only current date.

In [2]:
import requests
from bs4 import BeautifulSoup

Here we will get the main page from car section of bazos.

In [5]:
page = requests.get('https://auto.bazos.cz/')

In [4]:
soup = BeautifulSoup(page.text, 'html')
print(soup)

<!DOCTYPE html>
<html lang="cs">
<head>
<title>Auto bazar | Bazoš.cz</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Auto bazar zdarma. Vybírejte z 341 503 inzerátů. Prodej snadno a rychle na Bazoši. Přes půl milionů uživatelů za den. Najděte co potřebujete." name="description"/><link href="https://auto.bazos.cz/" rel="canonical"/><meta content="1055875657" property="fb:admins"/>
<link href="https://www.bazos.cz/bazos60s.css" rel="stylesheet" type="text/css"/><link as="image" href="https://www.bazos.cz/obrazky/bazos.svg" rel="preload"/>
<link href="https://www.bazos.cz/bazosprint.css" media="print" rel="stylesheet" type="text/css"/>
<link href="https://www.bazos.cz/favicon.ico" rel="shortcut icon"/>
<link href="https://www.bazos.cz/obrazky/icon-192x192.png" rel="icon" sizes="192x192"/>
<link href="https://www.bazos.cz/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://www.bazos.cz/rss.php?rub=au" rel="alternate" title="Auto RS

There are 20 adds plus other page elements. Lets pick just one add to see their construction.

In [30]:
soup.find('div', {'class':'inzeraty inzeratyflex'})

<div class="inzeraty inzeratyflex">
<div class="inzeratynadpis"><a href="/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php"><img alt="VOLKSWAGEN GOLF GTD 135kW, MANUÁL, VÝHŘEV, NAVI, LED" class="obrazek" height="105" src="https://www.bazos.cz/img/1t/412/151109412.jpg?t=1649934765" width="140"/></a>
<h2 class="nadpis"><a href="/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php">VOLKSWAGEN GOLF GTD 135kW, MANUÁL, VÝHŘEV, NAVI, LED</a></h2><span class="velikost10"> - <span class="ztop" title="TOP 21x Platí do 26.5. 2022">TOP</span> - [14.4. 2022]</span><br/>
<div class="popis">Volkswagen Golf - zachovalý vůz
GTD, 6-ti stupňová manuální převodovka, vyhřívaná sedadla, satelitní navigace, LED denní svícení

Do provozu 10/2013
Najeto: 161 936 km
Objem: 1968 ccm
Výkon: 135kW (183PS)
Druh paliva: nafta
Stav: perfektní
K ...</div><br/><br/>
</div>
<div class="inzeratycena"><b>  359 900 Kč</b></div>
<div class="inzeratylok">Brno<br/>620 00</div>
<div cla

There are several elements, which are important to us. In this section, we are interested in add id, which we are able to extract from href in element a on the second line. Its the number after "inzerat/". We are also interested in the href itself, since it is our link to follow to the actual add. Another element important to us is h2 class = "nadpis", which contains title of the add, from which we can extract type of car. And the last interesting element is span with class "velikost10", which reffers to date uploaded.

Now we can follow the link to the actual add page to see its content.

In [31]:
add_page = requests.get('https://auto.bazos.cz/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php')

In [33]:
soup_add = BeautifulSoup(add_page.text, 'html')

In [34]:
print(soup_add.find('div', {'class':'popisdetail'}))

<div class="popisdetail">Volkswagen Golf - zachovalý vůz<br/>
GTD, 6-ti stupňová manuální převodovka, vyhřívaná sedadla, satelitní navigace, LED denní svícení<br/>
<br/>
Do provozu 10/2013<br/>
Najeto: 161 936 km<br/>
Objem: 1968 ccm<br/>
Výkon: 135kW (183PS)<br/>
Druh paliva: nafta<br/>
Stav: perfektní<br/>
Karoserie: hatchback<br/>
Barva: červená<br/>
Počet míst: 5<br/>
Počet dveří: 5<br/>
<br/>
VIN: WVWZZZAUZEW112045<br/>
<br/>
Výbava:<br/>
<br/>
6 rychlostních stupňů; 6x airbag; ABS; adaptivní tempomat; alu kola; aut. klimatizace; autorádio; AUX; bi-xenonové světlomety; bluetooth; CD přehrávač; centrál dálkový; centrální zamykání; deaktivace airbagu spolujezdce; dělená zadní sedadla; denní svícení; dvouzónová klimatizace; el. okna; el. sklopná zrcátka; el. zrcátka; elektronická ruční brzda; hands free; imobilizér; isofix; LED denní svícení; manuální převodovka; mlhovky; multifunkční volant; nastavitelný volant; ostřikovače světlometů; palubní počítač; podélný posuv sedadel; polohov

Here we can see the description, which we will use to extract parameters of the car. This will be one of the most difficult parts of the project to extract the relevant text in unformated text. There is no official format to the text, so we will have to find a way how to extract this information from various formats.

After retrieving these informations, we will use analysis of the data got from this process.